# Import Library

In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import shutil
import pickle

# Function

In [2]:
Index2Label = {0: 'Background', 
               1: 'Head', 
               2: 'Torso', 
               3: 'Upper_Arms', 
               4: 'Lower_Arms', 
               5: 'Upper_Legs', 
               6: 'Lower_Legs'}

Label2Index = {'Background': 0, 
               'Head': 1, 
               'Torso': 2, 
               'Upper_Arms': 3, 
               'Lower_Arms': 4, 
               'Upper_Legs': 5, 
               'Lower_Legs': 6}

# Label2Index = {v: k for k, v in Index2Label.items()}

In [3]:
def create_images_path(folder_path):
    folder_path = folder_path
    imges = os.listdir(folder_path)
    images_path = [os.path.join(folder_path, img) for img in imges]
    return images_path

In [5]:
def palette_to_one_hot(palette_image_path, num_classes=7, imge_info=True, visualize=True):
    """
    팔레트 이미지를 원핫인코딩하여 (2940, 1960, 7) 형태의 이미지로 변환하는 함수

    Args:
    palette_image_path (str): 세그멘테이션된 P 모드의 PIL 팔레트 이미지 경로

    Returns:
    np.ndarray: 원핫인코딩된 이미지
    """
    # 클래스 수 정의 (배경 포함 1개)
    num_classes = num_classes

    # PIL 2 Numpy
    palette_image = Image.open(palette_image_path)
    palette_image_array = np.array(palette_image)

    # 원핫인코딩된 이미지를 저장할 배열 생성
    one_hot_encoded_image = np.zeros((*palette_image_array.shape, num_classes), dtype=np.uint8)

    # 각 픽셀을 원핫인코딩된 형태로 변환
    for i in range(palette_image_array.shape[0]): # 2940
        for j in range(palette_image_array.shape[1]): # 1960
            pixel_value = palette_image_array[i, j]  # 현재 픽셀의 값
            one_hot_encoded_image[i, j, pixel_value] = 1  # 해당 클래스에 대응하는 채널에 1 할당

    if imge_info:
        # 이미지 이름 출력
        image_name = os.path.basename(palette_image_path)
        print("Image Name:", image_name)
        print("Original Image Shape:", palette_image_array.shape)
        print("Transformed Image Shape:", one_hot_encoded_image.shape)
        print()
        
    if visualize:
        visualize_one_hot_encoded_image(one_hot_encoded_image,  Index2Label=Index2Label)
        
    return one_hot_encoded_image

In [4]:
# 각도를 이름으로 저장하기 위한 함수 수정

def ANGLE_save_images_pkl_with_one_hot_encoding(images_paths, output_folder_path, num_classes=7, imge_info=True, visualize=False):
    """
    이미지를 원핫인코딩하여 새로운 폴더에 저장하는 함수

    Args:
    images_paths (list): 이미지 파일 경로의 리스트
    output_folder_path (str): 이미지를 저장할 폴더 이름(현재 경로에 저장)
    num_classes (int): segmentation 세그멘테이션 클래스 개수(배경 포함)
    imge_info (bool): 이미지 정보 출력 토글(default: True)
    visualize (bool): 이미지 시각화 토글(default: False)

    Returns:
    None
    """
    # 새로운 폴더 생성
    os.makedirs(output_folder_path, exist_ok=True)
    
    # one_hot_encoded_images
    one_hot_encoded_images = []

    # 원핫 인코딩 수행
    for img_path in tqdm(images_paths, desc='Encoding images'):
        
        # PIL Pallete 2 Numpy
        one_hot_encoded_image = palette_to_one_hot(img_path, num_classes=num_classes, imge_info=imge_info, visualize=visualize)
        one_hot_encoded_images.append(one_hot_encoded_image)
        
    # 저장할 경로 설정
    save_path = os.path.join(output_folder_path, f"one_hot_encoded_images_array_{ang}.pkl") 
        
    # 이미지 데이터를 피클로 저장
    with open(save_path, 'wb') as f:
        pickle.dump(one_hot_encoded_images, f)

# Task 02.28.Wed

- Image Preprocessing: one hot encodding

In [7]:
base_folder_path = 'original_seg_crop'
# output_folder_path = 'one_hot_encoded_array_pkls'
# os.makedirs(output_folder_path, exist_ok=True)

In [ ]:
# angles = os.listdir(base_folder_path)

# for ang in angles:
#     angle_path = os.path.join(base_folder_path, ang) # 1~32
#     angle_imges_path = create_images_path(angle_path) # 각도별 촬영 이미지
    
#     print(f'Angle {ang} is processing.\n')
    
#     ANGLE_save_images_pkl_with_one_hot_encoding(angle_imges_path,
#                                                 output_folder_path,  
#                                                 num_classes=7, 
#                                                 imge_info=False, 
#                                                 visualize=False)
#     print(f'Angle {ang} is successly processed./n')

In [ ]:
# angles = os.listdir(base_folder_path)

# total_images = []

# for ang in angles:
#     print(f'Angle {ang} is processing.\n')  
#     angle_path = os.path.join(base_folder_path, ang) # 1~32
#     angle_imges_path = create_images_path(angle_path) # 각도별 촬영 이미지
    
#     angle_images = []
    
#     for image in angle_imges_path:
#         encoded_image = palette_to_one_hot(image, num_classes=7, imge_info=False, visualize=False)
#         angle_images.append(encoded_image)
        
#     total_images.append(angle_images)
    
#     print(f'Angle {ang} is successly processed./n')
   
   
#     print(len(total_images))
#     print(len(angle_images))
   
#     break


In [ ]:
angles = os.listdir(base_folder_path)

total_images = []
# 각도별 이미지 처리
for ang in tqdm(angles, desc='Processing Angles', unit='angle'):
    print(f'Angle {ang} is processing.\n')  
    angle_path = os.path.join(base_folder_path, ang)
    angle_images_path = create_images_path(angle_path)  # 각도별 촬영 이미지
    
    angle_images = []
    # 이미지별 원핫인코딩 및 처리
    for image in tqdm(angle_images_path, desc=f'Processing Images in Angle {ang}', unit='image', leave=False):
        encoded_image = palette_to_one_hot(image, num_classes=7, imge_info=False, visualize=False)
        angle_images.append(encoded_image)
        
    total_images.append(angle_images)
    
    print(f'Angle {ang} is successfully processed.\n')
    print(len(angle_images))
    
    # break
    
print(len(total_images))


In [13]:
# 배치 사이즈 설정
batch_size = 10

angles = os.listdir(base_folder_path)

print(angles)

total_images = []
# 각도별 이미지 처리
for ang in tqdm(angles, desc='Processing Angles', unit='angle'):
    print(f'Angle {ang} is processing.\n')  
    angle_path = os.path.join(base_folder_path, ang)
    angle_images_path = create_images_path(angle_path)  # 각도별 촬영 이미지
    
    angle_images = []
    # 이미지별 원핫인코딩 및 처리
    for i in tqdm(range(0, len(angle_images_path), batch_size), desc=f'Processing Images in Angle {ang}', unit='batch', leave=False):
        batch_images = angle_images_path[i:i+batch_size]
        batch_encoded_images = []
        for image in batch_images:
            encoded_image = palette_to_one_hot(image, num_classes=7, imge_info=False, visualize=False)
            batch_encoded_images.append(encoded_image)
        angle_images.extend(batch_encoded_images)
    
    total_images.append(angle_images)
    
    print(f'Angle {ang} is successfully processed.\n')
    print(len(angle_images))
    
    # break
    
print(len(total_images))

01


Processing Angles:   0%|          | 0/2 [00:00<?, ?angle/s]

Angle 0 is processing.



FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'original_seg_crop\\0'

In [16]:
base_folder_path = 'original_seg_crop'
# output_folder_path = 'one_hot_encoded_array_pkls'
# os.makedirs(output_folder_path, exist_ok=True)

# 배치 사이즈 설정
batch_size = 100

angles = os.listdir(base_folder_path)

total_images = []
# 각도별 이미지 처리
for ang in tqdm(angles, desc='Processing Angles', unit='angle'):
    print(f'Angle {ang} is processing.\n')  
    angle_path = os.path.join(base_folder_path, ang)
    angle_images_path = create_images_path(angle_path)  # 각도별 촬영 이미지
    
    angle_images = []
    # 이미지별 원핫인코딩 및 처리
    for i in tqdm(range(0, len(angle_images_path), batch_size), desc=f'Processing Images in Angle {ang}', unit='batch', leave=False):
        batch_images = angle_images_path[i:i+batch_size]
        batch_encoded_images = []
        for image in batch_images:
            encoded_image = palette_to_one_hot(image, num_classes=7, imge_info=False, visualize=False)
            batch_encoded_images.append(encoded_image)
        angle_images.extend(batch_encoded_images)
    
    total_images.append(angle_images)
    
    print(f'Angle {ang} is successfully processed.\n')
    print(len(angle_images))
    
    # break
    
print(len(total_images))

Processing Angles:   0%|          | 0/32 [00:00<?, ?angle/s]

Angle 01 is processing.



Processing Angles:   0%|          | 0/32 [01:50<?, ?angle/s]


KeyboardInterrupt: 

## Resize

In [24]:
base_folder_path = "original_seg_crop"
output_folder_path = 'one_hpt_encoded_array_pkls'
resize_seg_path = 'resized_seg'

In [ ]:
import os
from tqdm import tqdm
from PIL import Image

base_folder_path = "original_seg_crop"
output_folder_path = resize_seg_path

os.makedirs(output_folder_path, exist_ok=True)

angles = os.listdir(base_folder_path)

# 리사이즈할 이미지 크기 설정
target_height = 512
target_width = 512

for ang in tqdm(angles, desc='Processing Angles', unit='angle'):
    angle_path = os.path.join(base_folder_path, ang)
    angle_imges_path = create_images_path(angle_path)  # 각도별 촬영 이미지
    
    for image_path in angle_imges_path:
        image_name = os.path.basename(image_path)
        
        # 이미지 불러오기 (PIL 이미지로)
        image = Image.open(image_path)
        
        # 이미지 리사이즈
        resized_image = image.resize((target_width, target_height))
        
        # 저장할 경로 설정
        output_image_path = os.path.join(output_folder_path, ang, image_name)
        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
        
        # 이미지 저장
        resized_image.save(output_image_path)


In [28]:
angles = os.listdir(base_folder_path)

for ang in tqdm(angles, desc = 'Angle is processing'):
    angle_path = os.path.join(base_folder_path, ang) # 1~32
    angle_imges_path = create_images_path(angle_path) # 각도별 촬영 이미지
    
    print(len(angle_imges_path))
        
    for image_path in angle_imges_path:
        image_name = os.path.basename(image_path)
        
        
        
        
        
    
    

Angle is processing: 100%|██████████| 32/32 [00:00<00:00, 603.57it/s]

872
01_00_F001_01.png
894
01_00_F002_02.png
700
01_00_F002_03.png
705
01_00_F002_04.png
749
01_00_F003_05.png
680
01_00_F002_06.png
602
01_00_F003_07.png
674
01_00_F001_08.png
500
02_00_F001_09.png
573
02_00_F001_10.png
623
02_02_M001_11.png
555
02_00_F001_12.png
589
01_00_F001_13.png
713
01_00_F001_14.png
847
01_00_F003_15.png
825
01_00_F001_16.png
717
01_00_F001_17.png
872
01_00_F001_18.png
977
01_00_F001_19.png
908
01_00_F001_20.png
671
01_00_M002_21.png
549
01_00_F001_22.png
823
01_00_F001_23.png
858
01_00_F001_24.png
456
01_00_F003_25.png
542
01_00_F001_26.png
538
02_02_F395_27.png
585
02_00_M001_28.png
690
01_00_M001_29.png
710
01_00_M001_30.png
585
01_00_F003_31.png
600
01_00_F001_32.png


In [ ]:


os.makedirs(output_folder_path, exist_ok=True)

angles = os.listdir(base_folder_path)

for ang in angles:
    angle_path = os.path.join(base_folder_path, ang) # 1~32
    angle_imges_path = create_images_path(angle_path) # 각도별 촬영 이미지
    
    print(f'Angle {ang} is processing.\n')
    
    ANGLE_save_images_pkl_with_one_hot_encoding(angle_imges_path,
                                                output_folder_path,  
                                                num_classes=7, 
                                                imge_info=False, 
                                                visualize=False)
    print(f'Angle {ang} is successly processed./n')
    
    break


In [29]:
base_folder_path = "original_seg_crop"
output_folder_path = 'one_hpt_encoded_array_pkls'
resize_seg_path = 'resized_seg'

In [32]:
os.makedirs(output_folder_path, exist_ok=True)

angles = os.listdir(base_folder_path)

for ang in angles:
    angle_path = os.path.join(base_folder_path, ang) # 1~32
    
    angle_imges_path = create_images_path(angle_path) # 각도별 촬영 이미지
    
    
    # encoded_image = palette_to_one_hot

['original_seg_crop\\01\\01_00_F001_01.png', 'original_seg_crop\\01\\01_00_F002_01.png', 'original_seg_crop\\01\\01_00_F003_01.png', 'original_seg_crop\\01\\01_00_M002_01.png', 'original_seg_crop\\01\\01_01_F001_01.png', 'original_seg_crop\\01\\01_01_F002_01.png', 'original_seg_crop\\01\\01_01_F003_01.png', 'original_seg_crop\\01\\01_01_F004_01.png', 'original_seg_crop\\01\\01_01_F017_01.png', 'original_seg_crop\\01\\01_01_F018_01.png', 'original_seg_crop\\01\\01_01_F020_01.png', 'original_seg_crop\\01\\01_01_F022_01.png', 'original_seg_crop\\01\\01_01_F025_01.png', 'original_seg_crop\\01\\01_01_F027_01.png', 'original_seg_crop\\01\\01_01_F380_01.png', 'original_seg_crop\\01\\01_01_F381_01.png', 'original_seg_crop\\01\\01_01_F395_01.png', 'original_seg_crop\\01\\01_01_F396_01.png', 'original_seg_crop\\01\\01_01_F438_01.png', 'original_seg_crop\\01\\01_01_F439_01.png', 'original_seg_crop\\01\\01_01_F444_01.png', 'original_seg_crop\\01\\01_01_F446_01.png', 'original_seg_crop\\01\\01_01_F

In [ ]:
base_folder_path = 'original_seg_crop'
# output_folder_path = 'one_hot_encoded_array_pkls'
# os.makedirs(output_folder_path, exist_ok=True)

# 배치 사이즈 설정
batch_size = 100

angles = os.listdir(base_folder_path)

total_images = []
# 각도별 이미지 처리
for ang in tqdm(angles, desc='Processing Angles', unit='angle'):
    print(f'Angle {ang} is processing.\n')  
    angle_path = os.path.join(base_folder_path, ang)
    angle_images_path = create_images_path(angle_path)  # 각도별 촬영 이미지
    
    angle_images = []
    # 이미지별 원핫인코딩 및 처리
    for i in tqdm(range(0, len(angle_images_path), batch_size), desc=f'Processing Images in Angle {ang}', unit='batch', leave=False):
        batch_images = angle_images_path[i:i+batch_size]
        batch_encoded_images = []
        for image in batch_images:
            encoded_image = palette_to_one_hot(image, num_classes=7, imge_info=False, visualize=False)
            batch_encoded_images.append(encoded_image)
        angle_images.extend(batch_encoded_images)
    
    total_images.append(angle_images)
    
    print(f'Angle {ang} is successfully processed.\n')
    print(len(angle_images))
    
    # break
    
print(len(total_images))

In [2]:
import os

In [8]:
# 1번 촬영 각도에 대해서만 배체사이즈로 작업 수행

base_folder_path = r"D:\알파코\SCHP\Inspection_Seg\02_24_inspect\01"
output_path = ''

batch_size = 50

angle_images = os.listdir(base_folder_path)

angle_images.sort()


for i in range(0, len(angle_images), batch_size):
    batch_images = angle_images[i:i+batch_size]
    print(len(batch_images))


50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
22
